In [1]:
#Import modules
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import optimizers
from keras import models
from keras import layers

In [2]:
#Specify path for training, validation and testing data
train_dir = '/notebooks/train'
test_dir = '/notebooks/test'
validation_dir = '/notebooks/validation'

### Preprocess and augment images
Pre-processing is necessary to convert images to a tensor format that can be fed into model.
Data augmentation allows for manipulation of images to artificially increase the size of our training dataset and thus improve model performance.

In [3]:
#Instante trainig ImageDataGenerator object
#rescale to ensure that all pixel values are in range [0-1]
#rotation_range: allow for 40deg. random rotation of images
#width_shift_range & height_shift_range: fraction of total width and height that image can be shifted by
#shear_range: range in which can shear image
#zoom_range: range in which can zoom image
#horizontal_flip: allows for horizontal flip of image
#fill_mode: areas of image that fall outside of original boundaries are filled as so aaa|abcd|ddd, where | denotes a boundary
train_datagenerator = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

#Instantiate testing ImageDataGenerator object
#rescale to ensure that all pixel values are in range [0-1]
#no further data augmentation
test_datagenerator = ImageDataGenerator(rescale=1./255)

#Define function to generate augmented training dataset from training data directory
def train_prep(data_location, target_image_size):
    training_generator = train_datagenerator.flow_from_directory(
        data_location,
        target_size = target_image_size,
        batch_size = 20,
        class_mode = 'binary')
    return training_generator

#Define function to generate validation/testing dataset from data directory
def val_test_prep(data_location, target_image_size):
    val_test_generator = test_datagenerator.flow_from_directory(
        data_location,
        target_size = target_image_size,
        batch_size = 20,
        class_mode = 'binary')
    return val_test_generator

#Generate training data
train_generator = train_prep(train_dir, (150, 150))

#Generate validation data
validation_generator = val_test_prep(validation_dir, (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Instantiate pre-trained convolutional base
Here we will use VGG16 convolutional base

In [4]:
#Load weights learned from training on image net dataset
#Don't include top (will add our own dense classifier)
convolutional_base = VGG16(weights = 'imagenet', include_top = False)

### Add densely connected layers on top of convolutional base

In [5]:
#Create sequential model
my_model = models.Sequential()
#Add convolutional base
my_model.add(convolutional_base)
#Add Global Average Pooling layer to downscale parameter space
my_model.add(layers.GlobalAveragePooling2D())
#Add dense layers to learn classes and output binary predictions
my_model.add(layers.Dense(256, activation = 'relu'))
my_model.add(layers.Dense(1, activation = 'sigmoid'))

#Display summary of model architecture
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 14,846,273
Trainable params: 14,846,273
Non-trainable params: 0
_________________________________________________________________


### Freeze convolutional base

In [6]:
convolutional_base.trainable = False

### Compile model

In [7]:
my_model.compile(loss = 'binary_crossentropy',
                 optimizer = optimizers.RMSprop(learning_rate = 2e-5),
                 metrics = ['acc'])

### Train the densely connected layers on training dataset
This step is necessary to ensure that the model is making decent predictions before we unfreeze and fine-tune any of the convolutional base.
Fine-tuning the convolutional base without first training the densely connected top layers would result in large loss and a very difficult training process.

In [8]:
history_dense_top = my_model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/30
100/100 [==============================] - 17s 137ms/step - loss: 0.7404 - acc: 0.4995 - val_loss: 0.6902 - val_acc: 0.5410
Epoch 2/30
100/100 [==============================] - 9s 90ms/step - loss: 0.6778 - acc: 0.5660 - val_loss: 0.6564 - val_acc: 0.6440
Epoch 3/30
100/100 [==============================] - 9s 90ms/step - loss: 0.6539 - acc: 0.6520 - val_loss: 0.6290 - val_acc: 0.6830
Epoch 4/30
100/100 [==============================] - 9s 92ms/step - loss: 0.6335 - acc: 0.6830 - val_loss: 0.6002 - val_acc: 0.7520
Epoch 5/30
100/100 [==============================] - 9s 92ms/step - loss: 0.6077 - acc: 0.7410 - val_loss: 0.5766 - val_acc: 0.7770
Epoch 6/30
100/100 [==============================] - 9s 93ms/step - loss: 0.5951 - acc: 0.7390 - val_loss: 0.5569 - val_acc: 0.7840
Epoch 7/30
100/100 [==============================] - 9s 94ms/step - loss: 0.5822 - acc: 0.7475 - val_loss: 0.5391 - val_acc: 0.7920
Epoch 8/30
100/100 [==============================] - 9s 94ms/step 

In [10]:
#Evaluate model before fine tuning
test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
test_loss_, test_acc_ = my_model.evaluate(test_generator, steps=37)
print('test acc:', test_acc_)

Found 730 images belonging to 2 classes.
37/37 [==============================] - 2s 64ms/step - loss: 0.3630 - acc: 0.8562
test acc: 0.8561643958091736


Without fine-tuning the convolutional base, the model achieves 84.5% accuracy on the validation set.

### Unfreeze final 3 layers of convolutional base

In [11]:
#Display structure of convolutional base
convolutional_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

Therefore everything before block5 should remain frozen, and block5 will be unfrozen ready for fine-tuning.

In [12]:
#Unfreeze all convolutional layers in block5
convolutional_base.trainable = True

for layer in convolutional_base.layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False

In [13]:
#Compile model
my_model.compile(loss = 'binary_crossentropy',
                 optimizer = optimizers.RMSprop(learning_rate = 1e-5),
                 metrics = ['acc'])

### Finetune model

In [14]:
history_final = my_model.fit(
              train_generator,
              steps_per_epoch=100,
              epochs=100,
              validation_data=validation_generator,
              validation_steps=50)

Epoch 1/100
100/100 [==============================] - 10s 94ms/step - loss: 0.3761 - acc: 0.8365 - val_loss: 0.2556 - val_acc: 0.8850
Epoch 2/100
100/100 [==============================] - 9s 92ms/step - loss: 0.3068 - acc: 0.8670 - val_loss: 0.2151 - val_acc: 0.9050
Epoch 3/100
100/100 [==============================] - 9s 92ms/step - loss: 0.2655 - acc: 0.8945 - val_loss: 0.2059 - val_acc: 0.9040
Epoch 4/100
100/100 [==============================] - 9s 92ms/step - loss: 0.2565 - acc: 0.8955 - val_loss: 0.1956 - val_acc: 0.9100
Epoch 5/100
100/100 [==============================] - 9s 92ms/step - loss: 0.2331 - acc: 0.9005 - val_loss: 0.1762 - val_acc: 0.9300
Epoch 6/100
100/100 [==============================] - 9s 92ms/step - loss: 0.2155 - acc: 0.9050 - val_loss: 0.1739 - val_acc: 0.9280
Epoch 7/100
100/100 [==============================] - 9s 93ms/step - loss: 0.2019 - acc: 0.9175 - val_loss: 0.1746 - val_acc: 0.9180
Epoch 8/100
100/100 [==============================] - 9s 92m

### Evaluate performance of fine-tuned model on testing dataset

In [15]:
test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
test_loss, test_acc = my_model.evaluate(test_generator, steps=37)
print('test acc:', test_acc)

Found 730 images belonging to 2 classes.
37/37 [==============================] - 1s 23ms/step - loss: 0.3138 - acc: 0.9329
test acc: 0.932876706123352


After fine-tuning the convolutional base, the model achieves 93.3% accuracy on the validation set.